<a href="https://colab.research.google.com/github/linainsaf/Dual-Attention-Guided-Gaze-Target-Detection-in-the-Wild/blob/main/DAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pickle
import numpy as np
import cv2

In [28]:
def normalize (mat):
    max_value = np.max(mat)
    return mat/max_value

def Md (Id,N,gz):    
    r=16  # constante
    sigma=0.3 #constante
       
    Fd=Id-np.sum(np.sum(N))/N.size
    
    
    Mfront = np.maximum(Fd, np.zeros_like(Fd))
    Mmid = np.maximum(1-r*Fd**2, np.zeros_like(Fd))
    Mback = np.maximum(-Fd, np.zeros_like(Fd))
    
    if gz >-1 and gz<sigma:
        return Mfront
    elif gz >-sigma and gz<sigma:
        return Mmid
    elif gz >sigma and gz<1:
        return Mback

def Mf(H, G, I, alpha=6):
    """
        H     : (hx, hy) head position
        G     : (gx, gy) gaze target direction
        I     : image
        alpha : parameter to decide the angle of view
        
        return : FOV attention Map
    """
    
    FOV = np.zeros_like(I)
    
    for i in range(I.shape[0]):
        for j in range(I.shape[1]):
            
            theta   = np.arccos( ( (np.array([i,j])-H)*G )/( np.linalg.norm(np.array([i,j])-H)*np.linalg.norm(G)) )
            FOV[i,j] = np.max(1-((alpha*theta)/np.pi), 0)
    
    return FOV

In [53]:
# get depth maps  from pickle file
gaze_estimations = pickle.load( open( "drive/MyDrive/MLA/prediction_gazefollow.pkl", "rb" ) )
heads_poses = pickle.load( open( "drive/MyDrive/MLA/head_poses.pkl", "rb" ) )
depth_maps_imgs = pickle.load( open( "drive/MyDrive/MLA/depth_estimation_maps_gaze_follow.pkl", "rb" ) )
depth_maps_names = pickle.load( open( "drive/MyDrive/MLA/depth_estimation_maps_name_gaze_follow.pkl", "rb" ) )
heads = pickle.load( open( "drive/MyDrive/MLA/imgs_heads_gaze_follow.pkl", "rb" ) )

removed =  []
for i in range(len(heads)):
  if  heads[i].any():
      heads[i] = cv2.resize(heads[i], dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
  else : 
      removed.append(i)
print(removed)

[22, 2026]


In [54]:
del heads[removed[0]]
del heads[removed[1]-1]
del heads[0]


In [31]:
# build the heatmaps
# depth_maps -- depth maps
#imgs_heads -- head images
#gz 
#H -- head position
#G -- gx, gy
#I -- image
def DAM(depth_maps,imgs_heads, gz, H, G, I, alpha=6):
    
    for i in range (np.size(depth_maps)):    
        heat_maps=Md(depth_maps[i], imgs_heads[i], gz[i])*Mf(H[i], G[i], I[i], alpha) # on doit multiplier par le module FOV
    
    return heat_maps

In [39]:
gz = gaze_estimations.cpu().detach().numpy()[:,2]

In [32]:
H = heads_poses.cpu().detach().numpy()

In [62]:
np.shape(depth_maps_imgs[0])

(400, 300)

In [51]:
depth_maps_names

array(['00001001.pfm', '00001001.png'], dtype='<U12')

In [33]:
depth_maps = []
depth_maps.append(depth_maps_imgs[:1000])
depth_maps.append(depth_maps_imgs[1000:2000])
depth_maps.append(depth_maps_imgs[2000:3000])
depth_maps.append(depth_maps_imgs[3000:4000])

In [ ]:
heat_maps = DAM(depth_maps, heads, gz, H, I, alpha=6)